In [1]:
import torch
import torch.nn as nn
from torch.nn.parallel import DataParallel
import numpy as np
import os
import re
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"    # Free GPU
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

In [2]:
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf",
    cache_dir="/data5/home/sahilm/NLP_Project/Llama_2_7b_chat_hf"
    )

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", 
                                          cache_dir="/data5/home/sahilm/NLP_Project/Llama_2_7b_chat_hf"
                                         )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
if torch.cuda.device_count() > 1:
    model = DataParallel(model)
    print("Moj")
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Moj


DataParallel(
  (module): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(32000, 4096)
      (layers): ModuleList(
        (0-31): 32 x LlamaDecoderLayer(
          (self_attn): LlamaSdpaAttention(
            (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (rotary_emb): LlamaRotaryEmbedding()
          )
          (mlp): LlamaMLP(
            (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
            (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
            (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): LlamaRMSNorm()
          (post_attention_layernorm): LlamaRMS

In [4]:
model_without_parallel = model.module

In [5]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.tokenize import word_tokenize

def calculate_bleu_score(reference, candidate):
    # Tokenize reference and candidate strings
    reference_tokens = word_tokenize(reference.lower())
    candidate_tokens = word_tokenize(candidate.lower())
    # print(reference_tokens)
    # print([reference_tokens])
    # print(candidate_tokens)
    # Calculate BLEU score
    bleu_score = sentence_bleu([reference_tokens], candidate_tokens,smoothing_function=SmoothingFunction().method1)
    
    return bleu_score

# Example usage:
reference = "मैं क्रिकेट खेलना चाहता हूं"
candidate = "मैं क्रिकेट खेलना चाहता हूं"
score = calculate_bleu_score(reference, candidate)
print("BLEU Score:", score)


BLEU Score: 1.0


In [6]:
english_file = '/data5/home/sahilm/NLP_Project/Dataset/Dest/final_data/en-hi/train.en'
gujarati_file = '/data5/home/sahilm/NLP_Project/Dataset/Dest/final_data/en-hi/train.hi'

english_sentences = []
gujarati_sentences = []
total_sentences = 5000

with open(english_file, 'r') as file:
    for i, line in enumerate(file):
        if i >= total_sentences:
            break
        english_sentences.append(line)

with open(gujarati_file, 'r') as file:
    for i, line in enumerate(file):
        if i >= total_sentences:
            break
        gujarati_sentences.append(line)


print(len(english_sentences))
print(len(gujarati_sentences))

# total_sentences = 5000
# english_sentences = english_sentences[:total_sentences]
# gujarati_sentences = gujarati_sentences[:total_sentences]

english_sentences = [sentence.rstrip('\n') for sentence in english_sentences]
gujarati_sentences = [sentence.rstrip('\n') for sentence in gujarati_sentences]


5000
5000


In [7]:
# for i in range(3):
#     print(english_sentences[i])
#     print(gujarati_sentences[i])
#     print()

In [8]:
PERCENTILE = 97
print(np.percentile([len(x) for x in english_sentences], PERCENTILE))
print(np.percentile([len(x) for x in gujarati_sentences], PERCENTILE))

264.0
253.0


In [9]:
max_seq_length = 300

In [10]:
# for i in range(3):
#     print(english_sentences[i])
#     print(gujarati_sentences[i])
#     print()

In [11]:
from torch.utils.data import Dataset, DataLoader
class TextDataSet(Dataset):
    def __init__(self, english_sentences, gujarati_sentences):
        super().__init__
        self.english_sentences = english_sentences
        self.gujarati_sentences = gujarati_sentences
        
    def __len__(self):
        return len(self.english_sentences)
    
    def __getitem__(self,idx):
        return self.english_sentences[idx], self.gujarati_sentences[idx]
dataset = TextDataSet(english_sentences,gujarati_sentences)  

In [12]:
# len(dataset)

In [13]:
# dataset[22]

In [14]:
batch_size=150
train_loader = DataLoader(dataset,batch_size)
iterator = iter(train_loader)

In [15]:
# for batch in train_loader:
#     # Do something with the batch
#     # print(batch)
#     # print(len(batch))
#     # print(len(batch[0]))
#     # print(len(batch[1]))
#     for i in range(len(batch[0])):
#         print(batch[0][i])
#         print(batch[1][i])
#         print()
#         if i==2:
#             break
#     break

In [16]:
import transformers

pipeline = transformers.pipeline(
    "text-generation",
    model = model_without_parallel,
    tokenizer = tokenizer,
    torch_dtype = torch.float16,
    device = 0 if device.type == "cuda" else -1,
    batch_size=128,
    truncation=True
)

In [19]:
total_bleu_score = 0
max_bleu_score = 0
with open("zero_shot_Hindi_output.txt", 'w') as f:

    for batch in train_loader:
        # Do something with the batch
        # print(batch)
        # print(len(batch))
        # print(len(batch[0]))
        # print(len(batch[1]))
        for i in range(len(batch[0])):
            text = batch[0][i].strip()
            # text = "I want to play cricket"
            
            reference = batch[1][i].strip()
            # reference = "मैं क्रिकेट खेलना चाहता हूं"
            
            print("ENGLISH: " + text)
            print("ENGLISH: " + text, file=f)
            print("HINDI: " + reference)
            print("HINDI: " + reference, file=f)
            
            template = f"""Translate the following text to Hindi:\nText: {text}\nOutput: """    
            
            sequences = pipeline(
                template,
                do_sample = True,
                top_k = 10,
                num_return_sequences = 1,
                eos_token_id = tokenizer.eos_token_id,
                max_length = max_seq_length,
                truncation = True
            )
            output = sequences[0]['generated_text']

            sentences_after_output = re.findall(r'Output:(.*)', output)
            output = sentences_after_output[0].strip()
            print("OUTPUT: " + output)
            print("OUTPUT: " + output,file=f)
            bleu_score = calculate_bleu_score(reference, output)
            max_bleu_score = max(bleu_score,max_bleu_score)
            total_bleu_score += bleu_score
            print("BLEU Score:", bleu_score)
            print("BLEU Score:", bleu_score,file=f)
            print("Average BLEU Score:", total_bleu_score/(i+1))
            print("Average BLEU Score:", total_bleu_score/(i+1),file=f)
            print()
            print("",file=f)
            if i==2:
                break
        break

    print("Max BLEU Score: ", max_bleu_score)
    print("Max BLEU Score: ", max_bleu_score,file=f)

ENGLISH: In reply, Pakistan got off to a solid start.
HINDI: जिसके जवाब में पाक ने अच्छी शुरुआत की थी.
OUTPUT: पाकिस्तान का सुरकSHA सタर्प कर दिया.
BLEU Score: 0.02151810250197126
Average BLEU Score: 0.02151810250197126

ENGLISH: The European Union has seven principal decision-making bodies, its institutions: the European Parliament, the European Council, the Council of the European Union, the European Commission, the Court of Justice of the European Union, the European Central Bank and the European Court of Auditors.
HINDI: यूरोपीय संघ के महत्वपूर्ण संस्थानों में यूरोपियन कमीशन, यूरोपीय संसद, यूरोपीय संघ परिषद, यूरोपीय न्यायलय एवं यूरोपियन सेंट्रल बैंक इत्यादि शामिल हैं।
OUTPUT: एशिया यूनियन ने सप्ताह का समानुभूत पालित करने की सारी नियमाना संस्थान: ईश्वर पार्लिमेंट, ईश्वर कौंसल, संस्थान ऑफ ईश्वर उन्नायन, एसको जस्टिस ऑफ ईश्वर कान्सल, एसको जस्टिस ऑफ ईश्वर एक्सामा, एसको जस
BLEU Score: 0.006414921514603761
Average BLEU Score: 0.01396651200828751

ENGLISH: The Congress leader represents Siv